### Insert game ID and schedule information into my SQL database
Use the information located in the CSV files. For these inserts, we want the CSV files located in '../data/season_scores/'. When the insert has been completed and validated, it is okay to empty the CSV file. 

However, in the early stages of using MySQL, we should simply move the CSV files elsewhere (store on a hardrive, etc.) until we are sure that using MySQL is the correct path.

In [1]:
# Packages
import pandas as pd
import sqlalchemy

In [22]:
# User input
# Season
season = '2023'

# MySQL connection password
pword = ''

In [23]:
# Create the engine to connect to the MySQL database
engine = sqlalchemy.create_engine('mysql+mysqlconnector://root:' + pword + '@localhost/nhl')

In [24]:
# Read in the CSV file with the cleaned and scraped schedule information
sched = pd.read_csv('../data/season_scores/scores_' + season + '.csv')
display(sched.head())

,team,game_id,date,season,location,G,OT_status,win_flag
0,NSH,221007_NSHSJS,2022-10-07,2023,H,4,NaN,1
1,SJS,221008_SJSNSH,2022-10-08,2023,H,2,NaN,0
2,LAK,221011_LAKVGK,2022-10-11,2023,H,3,NaN,0
3,NYR,221011_NYRTBL,2022-10-11,2023,H,3,NaN,1
4,ANA,221012_ANASEA,2022-10-12,2023,H,5,OT,1


In [25]:
# Separate the table into data frames for the game ID and schedule MySQL tables
game_ids = sched[['team', 'date', 'game_id']]
sched.drop(columns='game_id', inplace=True)

In [26]:
# View the tables before inserting
display(game_ids.head())
display(sched.head())

,team,date,game_id
0,NSH,2022-10-07,221007_NSHSJS
1,SJS,2022-10-08,221008_SJSNSH
2,LAK,2022-10-11,221011_LAKVGK
3,NYR,2022-10-11,221011_NYRTBL
4,ANA,2022-10-12,221012_ANASEA


,team,date,season,location,G,OT_status,win_flag
0,NSH,2022-10-07,2023,H,4,NaN,1
1,SJS,2022-10-08,2023,H,2,NaN,0
2,LAK,2022-10-11,2023,H,3,NaN,0
3,NYR,2022-10-11,2023,H,3,NaN,1
4,ANA,2022-10-12,2023,H,5,OT,1


In [27]:
# Insert into game ID table
game_ids.to_sql(name = 'game_ids', con=engine, schema='nhl', if_exists='append', index=False)

2624

In [28]:
# Insert into schedule table
# This requires a slightly different method since I have a trigger set up to auto insert into scheudle table when a new record is recorded in the game ID table.
sched.to_sql(name='temp', con=engine, schema='nhl', if_exists='replace', index=False)

update_query = sqlalchemy.text("""
    UPDATE schedule
    INNER JOIN temp
        ON schedule.team = temp.team
        AND schedule.date = temp.date                      
    SET schedule.location = temp.location,
        schedule.G = temp.G,
        schedule.OT_status = temp.OT_status,
        schedule.win_flag = temp.win_flag;
""")

# Perform update
with engine.begin() as conn: 
    conn.execute(update_query)

#### Will currently need to go into MySQL to delete the temp table...